---
title: Pangeo X EarthCODE
subtitle: D.03.10 HANDS-ON TRAINING - EarthCODE 101 Hands-On Workshop - Example showing how to access data on the EarthCODE Open Science Catalog and working with the Pangeo ecosystem on EDC
authors:
  - name: Deyan Samardzhiev
    github: sunnydean
    orcid: 0009-0003-3803-8522
    affiliations:
      - id: Lampata UK
        institution: Lampata UK
reviewers:
  - name: Anne Fouilloux
    orcid: 0000-0002-1784-2920
    github: annefou
    affiliations:
      - id: Simula Research Laboratory
        institution: Simula Research Laboratory
        ror: 00vn06n10
date: 2025-06-01
thumbnail: https://raw.githubusercontent.com/ESA-EarthCODE/documentation/refs/heads/main/pages/public/img/EarthCODE_kv_transparent.png
keywords: ["earthcode", "pangeo", "stac", "xarray", "earth observation", "remote sensing"]
tags: ["pangeo"]
releaseDate: 2025-06-01
datePublished: 2025-06-01
dateModified: 2025-06-01
banner: ../static/PANGEO.png
github: https://github.com/sunnydean/LPS25_Pangeo_x_EarthCODE_Workshop
license: MIT
---

In [ ]:
# remove for EDC actual version, just for local testing
from distributed.client import _global_clients
for client in list(_global_clients.values()):
    client.close()


## Table of Content
```{contents}
:depth: 1
```

## Context
We will be using the [Pangeo](https://pangeo.io/) open-source software stack to demonstrate how to fetch EarthCODE published data and publically available satellite Sentinel-2 data to generate burn severity maps for the assessment of the areas affected by wildfires.

### Methodology approach
* Access Sentinel-2 L2A cloud optimised dataset through STAC
* Compute the Normalised Burn Ratio (NBR) index to highlight burned areas
* Classify burn severity

### Highlights
* The NBR index uses near-infrared (NIR) and shortwave-infrared (SWIR) wavelengths.


## Data
We will use Sentinel-2 data accessed via [element84's STAC API](https://element84.com/earth-search/) endpoint and the [SeasFire Data Cube](https://opensciencedata.esa.int/products/seasfire-cube/collection) to find burned areas, inspect them in more detail and generate burn severity maps for the assessment of the areas affected by wildfires.



#### Related publications
* https://www.sciencedirect.com/science/article/pii/S1470160X22004708#f0035
* https://github.com/yobimania/dea-notebooks/blob/e0ca59f437395f7c9becca74badcf8c49da6ee90/Fire%20Analysis%20Compiled%20Scripts%20(Gadi)/dNBR_full.py
* *Alonso, Lazaro, Gans, Fabian, Karasante, Ilektra, Ahuja, Akanksha, Prapas, Ioannis, Kondylatos, Spyros, Papoutsis, Ioannis, Panagiotou, Eleannna, Michail, Dimitrios, Cremer, Felix, Weber, Ulrich, & Carvalhais, Nuno. (2022). SeasFire Cube: A Global Dataset for Seasonal Fire Modeling in the Earth System (0.4) [Data set]. Zenodo. @alonso-2024. The same dataset can also be downloaded from Zenodo: https://zenodo.org/records/13834057*










### Packages

As best practices dictate, we recommend that you install and import all the necessary libraries at the top of your Jupyter notebook.

In [2]:
import json
import stackstac
import xarray
import pystac
import dask.distributed
import rasterio
from odc.stac import stac_load
import odc
import os
from odc.stac import configure_rio
from pystac.extensions.datacube import DatacubeExtension
from odc.stac import configure_s3_access, stac_load
from pystac.extensions.storage import StorageExtension
from datetime import datetime
from pathlib import Path

import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import requests
import xarray as xr
from ipyleaflet import Map, Polygon
from shapely import geometry
from shapely.geometry import shape, box
from ipyleaflet import Map, GeoJSON

from pystac_client import Client as pystac_client
from odc.stac import configure_rio, stac_load



# Load the Data

In [7]:
http_url = "https://s3.waw4-1.cloudferro.com/EarthCODE/OSCAssets/seasfire/seasfire_v0.4.zarr/"

ds = xarray.open_dataset(
	http_url,
	engine='zarr',
    chunks={},
	consolidated=True
	# storage_options = {'token': 'anon'}
)
ds

<xarray.Dataset> Size: 164GB
Dimensions:                         (latitude: 720, longitude: 1440, time: 966)
Coordinates:
  * latitude                        (latitude) float64 6kB 89.88 ... -89.88
  * longitude                       (longitude) float64 12kB -179.9 ... 179.9
  * time                            (time) datetime64[ns] 8kB 2001-01-01 ... ...
Data variables: (12/59)
    area                            (latitude, longitude) float32 4MB dask.array<chunksize=(180, 360), meta=np.ndarray>
    biomes                          (latitude, longitude) float32 4MB dask.array<chunksize=(180, 360), meta=np.ndarray>
    cams_co2fire                    (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    cams_frpfire                    (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    drought_code_max                (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    drought_code_mean               (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    ...                              ...
    t2m_max                         (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    t2m_mean                        (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    t2m_min                         (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    tp                              (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    vpd                             (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    ws10                            (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
Attributes:
    crs:          EPSG:4326
    description:  The SeasFire Cube is a scientific datacube for seasonal fir...
    title:        SeasFire Cube: A Global Dataset for Seasonal Fire Modeling ...

# Forest Fires Example Workflow

Search for a forest fire in Europe this last year

In [8]:
ds.gfed_ba

<xarray.DataArray 'gfed_ba' (time: 966, latitude: 720, longitude: 1440)> Size: 4GB
dask.array<open_dataset-gfed_ba, shape=(966, 720, 1440), dtype=float32, chunksize=(966, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude  (longitude) float64 12kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * time       (time) datetime64[ns] 8kB 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    aggregation:      Temporal | sum
    creator_notes:    Masked ocean with lsm variable. Missing years filled wi...
    description:      GFED4 dataset contains information about large fires only
    downloaded_from:  http://www.globalfiredata.org/data.html
    long_name:        Burned Areas from GFED
    missing years:    2016-2021
    provider:         Global Fire Emissions Database (GFED)
    units:            hectares (ha)

In [9]:
gwis = ds.gwis_ba
gwis

<xarray.DataArray 'gwis_ba' (time: 966, latitude: 720, longitude: 1440)> Size: 4GB
dask.array<open_dataset-gwis_ba, shape=(966, 720, 1440), dtype=float32, chunksize=(966, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude  (longitude) float64 12kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * time       (time) datetime64[ns] 8kB 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    aggregation:      Spatio-Temporal | sum
    creator_notes:    The missing data for the year 2001 has been filled with...
    downloaded_from:  https://gwis.jrc.ec.europa.eu/apps/country.profile/down...
    long_name:        Burned Areas from GWIS
    provider:         Global Wildfire Information System (GWIS)
    units:            hectares (ha)

In [10]:
with open("../aoi/feature.json") as f:
    feature = json.load(f)

poly = shape(feature["geometry"])
bbox = list(poly.bounds)

polygon = gpd.GeoDataFrame(index=[0], crs="epsg:4326", geometry=[geometry.box(*bbox)])
min_lon, min_lat, max_lon, max_lat = polygon.total_bounds

offset = 0
zoom = 1
# explore .where instead 

lat_start = gwis.latitude.sel(latitude=max_lat, method="nearest").item()   + offset*zoom
lat_stop  = gwis.latitude.sel(latitude=min_lat, method="nearest").item()   - offset*zoom
lon_start = gwis.longitude.sel(longitude=min_lon, method="nearest").item()  - offset*zoom  # better orientation
lon_stop  = gwis.longitude.sel(longitude=max_lon, method="nearest").item()  + offset*zoom


In [ ]:
## in SeasFire Datacube v2.0 the burned area variables would have the water bodies masked with ERA-5 land sea mask 
mask= ds['lsm']
mask

Find the biggest forest fire during the last three years within that place

'2018-01-01'


In [ ]:
gwis_aoi = gwis.sel(time=slice(datetime(2018,1,1),datetime(2021,1,1)), latitude=slice(lat_start, lat_stop),longitude=slice(lon_start, lon_stop))

In [ ]:
dateind_max_fire = gwis_aoi.sum(dim={'latitude','longitude'}).argmax(dim='time').compute().item()
fire_date = gwis_aoi.time[dateind_max_fire]
# where the sum in the plot is the highest

In [ ]:
# idxmax check out---> same thing with argmax + sel

In [ ]:
date_max_fire = gwis_aoi.isel(time=dateind_max_fire)
date_max_fire

In [ ]:
# gwis_all=gwis.resample(time="1Y").sum()
gwis_all=date_max_fire
gwis_all= gwis_all.where(mask)
gwis_all.plot()


In [ ]:
# import cartopy
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs



# da = gwis_all

# fig, ax = plt.subplots(
#     figsize=(8, 6),
#     subplot_kw={"projection": ccrs.PlateCarree()}
# )

# da.plot(
#     ax=ax,
#     transform=ccrs.PlateCarree(),
#     cmap="viridis",
#     cbar_kwargs={"label": "T₂ₘ (K)"}
# )

# ax.coastlines(resolution="10m", color="black", linewidth=1)
# ax.add_feature(cartopy.feature.BORDERS, linewidth=0.5)

# ax.set_extent([lon_start, lon_stop, lat_stop, lat_start], crs=ccrs.PlateCarree())

# plt.show()

In [ ]:
import hvplot.xarray

# Plot it interactively
gwis_all.hvplot(
    x='longitude',
    y='latitude',
    cmap='viridis',
    colorbar=True,
    frame_width=600,
    frame_height=400,
    geo=True,
    tiles='OSM'
)


In [ ]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


# lat_start=41
# lat_stop=41.5
# lon_start=-7.7
# lon_stop=-7.5

# lat_slice=slice(41.5,41)
# lon_slice=slice(-7.7,-7.5)

# fires = gwis_all.sel(latitude=lat_slice,longitude=lon_slice)
fire_date_t = pd.to_datetime(fire_date.values.item())

week_before = (fire_date_t - timedelta(days=7))
week_after = (fire_date_t + timedelta(days=7))

In [ ]:
print(week_before.date(), "---" , week_after.date())

In [ ]:
# from pystac_client import Client as pystac_client
# from odc.stac import configure_rio, stac_load


# Open a catalog
# catalog = pystac_client.open("https://earth-search.aws.element84.com/v1")
# time_start = "2020-07-01"
# time_end = "2020-07-28"
# time_range = time_start+"/"+time_end

In [ ]:
epsg

In [ ]:
index_name = 'NBR'

bandnames_dict = {
    'nir': 'nir',
    'swir22': 'swir22'
}
crs = "epsg:"+ str(epsg)  # projection on which the data will be projected

# km2deg = 1.0 / 111
# x, y = (23.9983519, 37.7351433)  # Center point of a query
# r = 4 * km2deg  
# bbox = (x - r, y - r, x + r, y + r)
# zoom = 1


# Normalised Burn Ratio, Lopez Garcia 1991
def calc_nbr(ds):
    return (ds.nir - ds.swir22) / (ds.nir + ds.swir22)

index_dict = {'NBR': calc_nbr}
index_dict

In [ ]:
catalog = pystac_client.open("https://earth-search.aws.element84.com/v1")


In [ ]:
zoom=1/2
chunk={}


In [ ]:
week_before_start = (week_before - timedelta(days=30))
time_range = str(week_before_start.date()) + "/" + str(week_before.date())

query1 = catalog.search(
    collections=["sentinel-2-l2a"], datetime=time_range, limit=100,
    bbox=bbox, query={"eo:cloud_cover": {"lt": 0.5}}
)

items = list(query1.get_items())
print(f"Found: {len(items):d} datasets")

items_pre = min(items, key=lambda item: item.properties["eo:cloud_cover"])

prefire_ds = stac_load(
    [items_pre],
    bands=("nir", "swir22"),
    # crs=crs,
    # resolution=  10*zoom,
    chunks=chunk,  # <-- use Dask
    groupby="datetime",
    bbox=bbox,
)
prefire_ds

In [ ]:
week_after_end = (week_after + timedelta(days=30))
time_range = str(week_after.date()) + "/" + str(week_after_end.date())

query2 = catalog.search(
    collections=["sentinel-2-l2a"], datetime=time_range, limit=100,
    bbox=bbox, query={"eo:cloud_cover": {"lt": 0.5}}
)

items = list(query2.get_items())
print(f"Found: {len(items):d} datasets")

items_post = min(items, key=lambda item: item.properties["eo:cloud_cover"])

postfire_ds = stac_load(
    [items_post],
    bands=("nir", "swir22"),
    # crs=crs,
    # resolution=10 * zoom,
    chunks=chunk,  # <-- use Dask
    groupby="datetime",
    bbox=bbox,
)
postfire_ds

In [ ]:
# Rename bands in dataset to use simple names 
bands_to_rename = {
    a: b for a, b in bandnames_dict.items() if a in prefire_ds.variables
}

# prefire
prefire_ds[index_name] = index_dict[index_name](prefire_ds.rename(bands_to_rename) / 10000.0)

# postfire
postfire_ds[index_name] = index_dict[index_name](postfire_ds.rename(bands_to_rename) / 10000.0)


In [ ]:
# calculate delta NBR
prefire_burnratio = prefire_ds.NBR.isel(time=0)
postfire_burnratio = postfire_ds.NBR.isel(time=0)

delta_NBR = prefire_burnratio - postfire_burnratio

dnbr_dataset = delta_NBR.to_dataset(name='delta_NBR').persist()

In [ ]:
dnbr_dataset
delta_NBR

In [ ]:
fig = plt.figure(1, figsize=[7, 10])

# We're using cartopy and are plotting in PlateCarree projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
#ax.set_extent([-180, 180, -70, 70], crs=ccrs.PlateCarree()) # lon1 lon2 lat1 lat2
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True)

# We need to project our data to the new Orthographic projection and for this we use `transform`.
# we set the original data projection in transform (here Mercator)
prefire_burnratio.plot(ax=ax, transform=ccrs.epsg(prefire_burnratio.spatial_ref.values), cmap='RdBu_r',
                       cbar_kwargs={'orientation':'horizontal','shrink':0.95})

# One way to customize your title
plt.title( pd.to_datetime(prefire_burnratio.time.values.item()).strftime("%d %B %Y"), fontsize=18)

In [ ]:
fig = plt.figure(1, figsize=[7, 9])

# We're using cartopy and are plotting in PlateCarree projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
#ax.set_extent([-180, 180, -70, 70], crs=ccrs.PlateCarree()) # lon1 lon2 lat1 lat2
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True)

# We need to project our data to the new Orthographic projection and for this we use `transform`.
# we set the original data projection in transform (here Mercator)
postfire_burnratio.plot(ax=ax, transform=ccrs.epsg(postfire_burnratio.spatial_ref.values), cmap='RdBu_r',
                        cbar_kwargs={'orientation':'horizontal','shrink':0.95})

# One way to customize your title
plt.title( pd.to_datetime(postfire_burnratio.time.values.item()).strftime("%d %B %Y"), fontsize=18)

In [ ]:
fig = plt.figure(1, figsize=[7, 10])

# We're using cartopy and are plotting in PlateCarree projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True)

# We need to project our data to the new Orthographic projection and for this we use `transform`.
# we set the original data projection in transform (here Mercator)
dnbr_dataset.delta_NBR.plot(ax=ax, transform=ccrs.epsg(dnbr_dataset.delta_NBR.spatial_ref.values), cmap='RdBu_r',
                            cbar_kwargs={'orientation':'horizontal','shrink':0.95})

# One way to customize your title
plt.title( "Delta NBR", fontsize=18)

https://un-spider.org/advisory-support/recommended-practices/recommended-practice-burn-severity/in-detail/normalized-burn-ratio

![img](https://un-spider.org/sites/default/files/table+legend.PNG)

In [ ]:
BURN_THRESH = 0.27
burn_mask = dnbr_dataset.delta_NBR > BURN_THRESH           # True/False mask, same shape as raster
burn_mask.plot()

In [ ]:
dx, dy = dnbr_dataset.delta_NBR.rio.resolution()
pixel_area_ha = abs(dx * dy) / 1e4       # 10m × 10m  → 0.01 ha
pixel_area_ha

pixels_burned   = burn_mask.sum().compute().item()   # integer number of burned pixels
burned_area_ha  = pixels_burned * pixel_area_ha

print(f"Pixels burned : {pixels_burned:,d}")
print(f"Burned area   : {burned_area_ha:,.2f} ha")
print(f"Actual Burned Area : {gwis_all.sum().compute():,.2f}, ha")

In [ ]:
dnbr_dataset['burned_ha_mask'] = burn_mask

In [ ]:
dnbr_dataset = dnbr_dataset.persist()
gwis_all = gwis_all.persist()


# r_dnbr_dataset = dnbr_dataset.rename({'x': 'lon', 'y': 'lat'})
# r_gwis_all = gwis_all.rename({'longitude': 'lon', 'latitude': 'lat'})

In [ ]:
gwis_all = gwis_all.rio.write_crs(ds.rio.crs)

Plot is off because of bad projection (curcilinear to rectilinear) but we can see that generally the fires are in the north-west/north-east regions with two distinct occurances

In [ ]:
import hvplot.xarray

gwis_all_reprojected = gwis_all.rio.reproject(dnbr_dataset.delta_NBR.rio.crs)

dnbr_plot = dnbr_dataset.delta_NBR.hvplot(
    width=700,
    height=700,
    title='dNBR (10 m) with GWIS overlay',
    alpha=1.0
)

# Plot the reprojected coarse dataset as transparent overlay
gwis_plot = gwis_all_reprojected.hvplot(
    cmap='Reds',
    alpha=0.3,
    clim=(0, gwis_all.max().compute().item())
)

# Combine them interactively
combined_plot = dnbr_plot * gwis_plot

combined_plot


# Saving Your Work

xrlint and validate your cube...

In [ ]:
import xrlint.all as xrl

linter = xrl.new_linter("recommended")
linter.validate(dnbr_dataset)


In [ ]:
# Assign dataset-level attributes
dnbr_dataset.attrs.update({
    'title': 'Delta NBR and Burned Area Mask Dataset',
    'history': 'Created by reprojecting and aligning datasets for fire severity analysis',
    'Conventions': 'CF-1.7'
})


# Assign variable-level attributes for delta_NBR
dnbr_dataset.delta_NBR.attrs.update({
    'institution': 'Lampata',
    'source': 'Sentinel-2 imagery; processed with open-source dNBR code, element84...',
    'references': 'https://example.com/ref',
    'comment': 'dNBR values represent change in vegetation severity post-fire',
    'standard_name': 'difference_normalized_burn_ratio',
    'long_name': 'Differenced Normalized Burn Ratio (dNBR)',
    'units': 'm'
})

# Example for burned_ha_mask data variable
dnbr_dataset.burned_ha_mask.attrs.update({
    'standard_name': 'burned_area_mask',
    'long_name': 'Burned Area Mask in Hectares',
    'units': 'hectares',
    'institution': 'Your Institution Name',
    'source': 'Derived from wildfire impact analysis',
    'references': 'https://example.com/ref',
    'comment': 'Burned area mask showing presence of burned areas'
})


In [ ]:
from xcube.core.verify import assert_cube

assert_cube(dnbr_dataset)  # raises ValueError if it's not xcube-valid


In [ ]:
# add time
dnbr_dataset = dnbr_dataset.expand_dims(time=[postfire_ds.time.isel(time=0).values])
dnbr_dataset

Chunk Data for Better Usability

In [ ]:
dnbr_dataset = dnbr_dataset.chunk({"time": 1, "y": 1000, "x": 1000})


In [ ]:
print(type(dnbr_dataset.burned_ha_mask.data))

In [ ]:
import os
save_at_folder = './wildfires'
if not os.path.exists(save_at_folder):
    os.makedirs(save_at_folder)

# Define the output path within your notebook folder
output_path = os.path.join(save_at_folder, "dnbr_dataset.zarr")

# save
dnbr_dataset.to_zarr(output_path, mode="w")